<a href="https://colab.research.google.com/github/m4vic/neuralnets_Pytorch/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

flow

tabular(1,784) --> reshape(28,28) it will become 2d --> 2d matrix --> change datatype --> Np.unit8 ---> ---> 1d to 3d --> (3,28,28) --> PIl image --> resize (3,256,256)  --> center crop (3,224,224) ---> Pytorch tensor scale (0-1) --> normalize  

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [4]:
#set random seeds for reproducibility
torch.manual_seed(42)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using device{device}")

using devicecpu


In [6]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head()


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df.shape

(60000, 785)

In [8]:
X = df.iloc[:, 1:].values # all from 1 index to end
y = df.iloc[:, 0].values # all 0 th index ie label

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Transformations

In [10]:
from torchvision import transforms
# defining the transforms


custom_transform = transforms.Compose([
    transforms.Resize(256), # resize 256
    transforms.CenterCrop(224),# center crop 224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # normalize
])

In [11]:
from PIL import Image
import numpy as np


class CustomDataset(Dataset):
    def __init__(self, features, labels, transform):
      self.features = features
      self.labels = labels
      self.transform = transform
    def __len__(self):
      return len(self.features)



    def __getitem__(self, idx):
      # resize to 28,28

      image = self.features[idx].reshape(28,28) # image will be the resizing the features to (28,28)

      # chainging the data type np.uint8

      image = image.astype(np.uint8)

      # convert blackwhite image to color

      image = np.stack([image]*3, axis=-1) # single channel to 3 channel and axis = -1 change the last one to 1st and 1st to last

      # convert step array to PIL image

      image = Image.fromarray(image)

      # apply transform

      image = self.transform(image) #

      #return

      return image, torch.tensor(self.labels[idx], dtype=torch.long) # returning the labels with feature image and converting the dtype to long



In [12]:
train_dataset = CustomDataset(x_train, y_train, custom_transform)
test_dataset = CustomDataset(x_test, y_test, custom_transform)


In [13]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True , pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

# *Using PreTrained Model VGG16*

In [16]:
# fetch the pretrained model

import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

In [17]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [18]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [19]:
# freezung the features weights

for param in vgg16.features.parameters():
  param.requires_grad = False

In [ ]:
#replacing the classifier of vgg to our own
vgg16.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=1024),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=1024, out_features=512),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(512,10)
)

In [ ]:
vgg16 = vgg16.to(device)

In [20]:
learning_rate = 0.0001
epochs = 5


In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [ ]:
# training loop

#training loop

for epoch in range(epochs):
  total_epoch_loss = 0
  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    #forward pass
    outputs = vgg16(batch_features)

    #calculate loss
    loss = criterion(outputs, batch_labels)

    #backward pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    # accumulate loss
    total_epoch_loss = total_epoch_loss +  loss.item()
  avg_loss = total_epoch_loss / len(train_loader)
  print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss}')

KeyboardInterrupt: 

In [ ]:
# set model to eval

vgg16.eval() # tell the model that we are going to eval

In [ ]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:

     # move data to gpu
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)


    outputs = model(batch_features)

    _, predicted = torch.max(outputs.data, 1) # give max from the predited value

    total = total + batch_labels.shape[0] #
    correct += (predicted == batch_labels).sum().item() # corrected if pred == batch

accuracy = correct / total

print(f'Accuracy: {accuracy}')

In [ ]:
total = 0
correct = 0

with torch.no_grad():
  for batch_features, batch_labels in train_loader:

     # move data to gpu
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)


    outputs = model(batch_features)

    _, predicted = torch.max(outputs.data, 1) # give max from the predited value

    total = total + batch_labels.shape[0] #
    correct += (predicted == batch_labels).sum().item() # corrected if pred == batch

accuracy = correct / total

print(f'Accuracy: {accuracy}')

# *ResNet TransferLearning*

In [22]:
resnet50 = models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 135MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [30]:
resnet50.fc

Linear(in_features=2048, out_features=1000, bias=True)